# Get Familiar with LLMs and Langchain

- 9기 조의현

Get Gemini Pro API: (Free usage!)

https://ai.google.dev/?gad_source=1&gclid=EAIaIQobChMI077Gkt_AhAMVNO4WBR2ByAoREAAYASAAEgJTdvD_BwE

Langchain Official Docs.

https://api.python.langchain.com/en/latest/langchain_api_reference.html

In [ ]:
!pip install -qq langchain google-generativeai langchain-google-genai

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [ ]:
## 프롬프트에 엔지니어링에 쓸 LLM 모델 설정 (Gemini Pro API 사용!)
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             google_api_key = "####", ## 본인 API 사용 (외부 유출 주의!!!)
                             convert_system_message_to_human = True)

# LangChain으로 Dataset Generation 수행하기

AI 관련 프로젝트를 진행하며 맞닥뜨릴 수 있는 난관 중 하나는, 원하는 작업을 수행하기 위해 모델을 학습시키고자 하는데, 모델이 학습할 수 있는 license-free 데이터셋이 충분하지 않다는 점입니다. 특히 AI 윤리가 뜨거운 화제로 떠오른 현 시점에서 양질의 오픈소스 데이터셋을 찾기가 더더욱 힘들어지고 있습니다.

따라서 LLM의 뛰어난 생성 능력을 힘입어 resource가 scarce한 domain에 대해 데이터셋을 augment하는 방면에서 연구가 활발하게 이루어지고 있습니다. Gemini-Pro는 아직 완벽한 수준의 데이터를 생성하지 못하지만, 저희가 무료로 사용할 수 있다는 점에서 이번 실습 때 사용하여 데이터셋을 생성하고자 합니다.

아래 예시는 어느 문단에 대해 두 사람 `phi`와 `Theta`간의 대화 형식의 토론 데이터셋을 LLM을 통해 생성하는 예시입니다. 모델링 프로젝트를 진행할 때 관련 데이터셋이 부족하거나 없을 시 해당 방법을 이용해 generate할 수 있습니다.

In [ ]:
## 본문

passage = """Cristiano Ronaldo is considered one of the greatest footballers in the history of the sport due to his remarkable records,
exceptional skills, and passion for the game. He has achieved success across various clubs, securing numerous titles in Europe's major leagues.
Ronaldo has shone at clubs like Manchester United, Real Madrid, and Juventus, leading them to league victories, Champions League triumphs, and more.
Among his records, he is the all-time leading scorer in the Champions League. Additionally, representing the Portugal national team,
he has guided them to victories in the UEFA European Championship and the Nations League.
Ronaldo's playing style is characterized by his outstanding scoring ability, speed, technique, and aerial prowess.
His relentless desire for self-improvement and focus on the game have cemented his status as a football legend."""

In [ ]:
## LLM에 넣어 원하는 결과를 얻을 때 받기를 원하는 형식을 정합니다.

format_instructions = """The output should be a markdown code snippet formatted in the following schema,
주제는 <topic></topic> 태그로 둘러싸여야 합니다.
토론은 <debate></debate> 태그로 둘러싸여야 합니다.
토론의 참가자는 각각 <phi></phi>, <theta></theta> 태그로 둘러싸어야 합니다."""

In [ ]:
## Prompt입니다. 앞서 정의한 본문, format_instruction을 넣고,
## 원하는 결과가 나오도록 유도하기 위해 여러 instructions을 추가합니다.

template = """
다음의 글을 고려해주세요: <passage>{passage}</passage>
{format_instructions}
두 가지 작업이 있습니다:
1) 글의 내용을 기반으로 호날두에 대해 하나의 토론 주제를 생성하세요.
생성된 토론 주제는 글의 내용과 일치해야 합니다.
생성된 토론 주제는 철학적이고, 창의적이며, 흥미롭고, 참여를 유도하며, 사고를 자극하는 것이어야 합니다.
생성된 토론 주제는 쉽게 답할 수 없으며, 양쪽에서 논의할 수 있어야 합니다.
2) 생성된 주제에 대해 두 합리적인 개인, Phi와 Theta 사이의 토론을 생성하세요.
토론에서 참가자들은 상호 반대되는 견해를 가지고 있어야 합니다.
토론에서 참가자들은 결코 자신을 반복해서는 안 됩니다.
토론에서 참가자들은 때때로 양보를 할 수 있지만, 여전히 자신의 관점을 견지해야 합니다.
토론에서 참가자들은 합리적인 토론의 다양한 기술을 사용하며, 감정적인 조작 기술을 사용하지 않아야 합니다.
"""

In [ ]:
## 결과 출력

prompt = PromptTemplate(
    input_variables = ["passage"],
    partial_variables={"format_instructions": format_instructions},
    template = template
)

_input = prompt.format(passage = passage)
output = llm.invoke(_input)

input_str = output.content

print(input_str)

In [ ]:
## 결과 전처리
import pandas as pd
import re

# topic, phi, theta 추출
topic = re.search(r'<topic>(.*?)</topic>', input_str).group(1).strip()
phis = re.findall(r'<phi>(.*?)</', input_str)
thetas = re.findall(r'<theta>(.*?)</', input_str)

# 최대 길이에 맞추어 리스트 길이를 맞춤
max_len = max(len(phis), len(thetas))
phis += [''] * (max_len - len(phis))
thetas += [''] * (max_len - len(thetas))

# DataFrame 생성
data = {'topic': [topic] * max_len,
        'phi': phis,
        'theta': thetas}

df = pd.DataFrame(data)

In [ ]:
df

# Langchain으로 사칙연산 문제풀이 챗봇 구축하기

Langchain의 Prompt Engineering 기능을 통해 자신이 원하는 결과를 출력할 수 있을 뿐더러, 챗지피티가 수식 계산, 코드 출력 등을 수행할 때 사용하는 `Code Interpreter`기능을 사용해 언어 모델이 수행하기 어려운 사칙 연산 등 작업을 수행할 수 있습니다. 해당 기능은 기본적으로 제공되는 tool 이외에도 자신이 정의한 python class을 적용할 수 있습니다.

아래 예시는 python 환경에서 계산을 수행한 후 다시 반환하는 아주 간단한 구현입니다. langchain agent는 현재 활용이 활발하게 이루어지고 있으므로 LLM을 공부하신다면 이 쪽을 중점으로 살펴보시기를 추천합니다.

In [ ]:
## langchain이 사용할 수 있는 python tool (code interpreter) 설정
## 사칙연산 식이 input으로 주어지면 해당 tool을 불러와 계산을 수행합니다.
from langchain.tools import BaseTool

class EvaluateMathExpression(BaseTool):
    name = "Math Evaluation"
    description = 'use this tool to evaluate a math expression.'

    def _run(self, expr: str):
        return eval(expr) ## python 환경에서 사칙연산 수행

    def _arun(self, query: str):
        raise NotImplementedError("Async operation not supported yet")

tools = [EvaluateMathExpression()]

In [ ]:
## LLM 모델에 persona를 부여하는 공식 PREFIX을 사용합니다.
from langchain.agents.conversational_chat.prompt import (PREFIX)

print(PREFIX)

In [ ]:
## LLM 모델이 사칙연산을 수행할 때 저희가 설정한 code interpreter를 사용하도록
## prompt 마지막에 제약조건을 하나 걸어줍니다.
prompt = PREFIX + "\n" + '''
Unfortunately, Assistant is terrible at maths. Assistant should always refers to available tools and never try to answer math questions by itself
'''

print(prompt)

In [ ]:
## Langchain Agent(모델)을 설정합니다.
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools, ## 앞서 정의한 tool 사용
    llm=llm, ## 앞서 정의한 Gemini Pro 사용
    verbose=True, ## langchain 연산 과정 보이게
    max_iterations=3,
    early_stopping_method='generate',
    memory=ConversationBufferWindowMemory(
        memory_key='chat_history',
        k=5,
        return_messages=True
    )
)

In [ ]:
## 사전에 정의한 프롬프트 Langchain Agent에 적용

customized_prompt = agent.agent.create_prompt(
    system_message = prompt,
    tools=tools
)

agent.agent.llm_chain.prompt = customized_prompt

In [ ]:
## Agent 실행

result = agent.invoke(f"Please solve this question : What is 2 + 2 * 0.13 - 1.001?")

In [ ]:
result['input']

In [ ]:
result['output']

In [ ]:
## 실제 계산 결과
2 + 2 * 0.13 - 1.001

In [ ]:
## 이전 history 불러오기
agent.memory.buffer